# Ranch Demo Notebook

This notebook has examples of how to make Ranch calls to run through the complete network building pipeline to develop standard roadway and transit network from scratch

In [1]:
import os
import pickle
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import json
import sys

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

In [2]:
%load_ext autoreload
%autoreload 2

# Remote I/O

User to update, project directory is not necessarily the ranch directory

In [3]:
# project directory
root_dir = os.path.join("D:/merced")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

# software directory
ranch_dir = os.path.join("D:/github/Ranch")

# the folder where SharedStreets extractions live

shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")

# the folder where OSMNX extractions live

osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

In [4]:
parameters = Parameters(ranch_base_dir = ranch_dir)

2022-01-07 16:51:41, INFO: Lasso base directory set as: D:/github/Ranch


In [5]:
parameters.standard_crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
for d in [root_dir, external_dir, interim_dir, shst_extract_dir, osm_extract_dir]:
    if os.path.isdir(d):
        print(f"{d} exists!")
        continue
    
    print(f"creating {d}")
    os.makedirs(d)

D:/merced exists!
D:/merced\data\external exists!
D:/merced\data\interim exists!
D:/merced\data\external\sharedstreets_extract exists!
D:/merced\data\external\osmnx_extract exists!


## Step 1 - SharedStreets Extraction

Extracts sharedstreets representation of OSM links.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from SharedStreets API.

In [7]:
# the polygon file for the area

input_polygon_file = os.path.join(
    external_dir,
    "sharedstreets_extract", 
    "merced.shp"
)

In [9]:
# the SharedStreets extraction file is saved to the 'output_dir' argument

ranch.run_shst_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = shst_extract_dir
)

epsg:4326
2022-01-07 16:52:28, INFO: Exporting boundry file D:/merced\data\external\sharedstreets_extract\boundary.0.geojson
2022-01-07 16:52:28, INFO: extracting for polygon 0


## Step 2 - OSMNX Extraction

Extracts complete OSM attributes using OSMNX.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from OSMNX.

In [10]:
# the OSMNX extraction file is saved to the 'output_dir' argument

ranch.run_osmnx_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = os.path.join(external_dir, "osmnx_extract")
)

## Step 3 - Consolidate SharedStreets and OSMNX extractions

consolidates raw roadway data and create initial roadway networks

In [16]:
# build and returns roaday network object from extraction files

roadway_network = Roadway.create_roadway_network_from_extracts(
    shst_extract_dir = shst_extract_dir,
    osm_extract_dir = osm_extract_dir,
    parameters = parameters,
)

2022-01-07 17:07:08, INFO: Reading sharedstreets data
2022-01-07 17:07:08, INFO: ----------start reading shst extraction data-------------
2022-01-07 17:07:08, INFO: reading shst extraction data : D:/merced\data\external\sharedstreets_extract\extract.boundary.0.out.geojson
2022-01-07 17:07:20, INFO: ----------finished reading shst extraction data-------------
2022-01-07 17:07:20, INFO: Removing duplicates in shst extraction data
2022-01-07 17:07:20, INFO: ...before removing duplicates, shst extraction has 52448 geometries.
2022-01-07 17:07:20, INFO: ...after removing duplicates, shst extraction has 52448 geometries.
2022-01-07 17:07:20, INFO: Reading osmnx data
2022-01-07 17:08:16, INFO: Extracting corresponding osm ways for every shst geometry
2022-01-07 17:09:13, INFO: shst extraction has 52448 geometries
2022-01-07 17:09:13, INFO: shst extraction has 55919 osm links
2022-01-07 17:09:13, INFO: ---joining osm shst with osmnx data---
2022-01-07 17:09:13, INFO: shst extraction has 52946

In [12]:
RanchLogger.info("Initial network has {} links".format(roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(roadway_network.shapes_df.shape[0]))

2022-01-07 17:01:39, INFO: Initial network has 102371 links
2022-01-07 17:01:39, INFO: Initial network has 38693 nodes
2022-01-07 17:01:39, INFO: Initial network has 52441 shapes


### Optional: If the user wants to write out the roadway network in standard format, they can do the following, otherwise no need to write out

In [11]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step3_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step3_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

2022-01-04 23:12:51, INFO: write out shape geojson
2022-01-04 23:13:09, INFO: write out node geojson
2022-01-04 23:13:21, INFO: write out link json
2022-01-04 23:13:25, INFO: write out link geojson


### Optional: it's more efficient if the user writes out the intermediate network object in pickle

In [12]:
working_network_filename = os.path.join(interim_dir,"step3_network.pickle")
pickle.dump(roadway_network, open(working_network_filename, 'wb'))

## Step 4 - Third-Party Roadway Data Conflation

Conflates e.g. county network with roadway network using SharedStreets.
The input for this step is the 3rd-party geodatabases to conflate.
The output of this step is the SharedStreets conflation outputs in geojson.

In [17]:
# define the input 3rd-party file, either shapefile or geojson

input_network_file = os.path.join(external_dir, "sjmodel", "Network", "2015","TCM_MASTER_2A22_102717.shp")

In [18]:
# if the CRS is undefined in the input network file, user needs to specify:

input_crs = CRS("ESRI:102643")

In [19]:
# the output is saved to the 'output_dir' argument
# user can specify the match option (reference here) 
# by 'custom_match_option'
# if not specified, use the default
# match option would vary by the input network

ranch.run_shst_match(
    input_network_file = input_network_file,
    input_crs = input_crs,
    output_dir = os.path.join(external_dir, "sjmodel", "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

2022-01-04 16:18:56, INFO: input network D:/sanjoaquin\data\external\sjmodel\Network\2015\TCM_MASTER_2A22_102717.shp has crs : None
2022-01-04 16:18:58, INFO: Input network for shst match does not have unique IDs, generating unique IDs
2022-01-04 16:18:58, INFO: Generated 49910 unique IDs for 49910 links in the input network
2022-01-04 16:18:58, INFO: Exporting shst match input - ID-ed geometry file D:/sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.geojson
2022-01-04 16:19:02, INFO: Exporting ID-ed network file D:/sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.full.geojson


## Step 5 - Tidy Up Roadway

Identifies drive dead-ends, cul-de-secs, numbering links and nodes, etc.
The input for this step is the polygon file with subregion identifier, e.g. county.
This step labels each link and node with the county name, and assign model IDs for links and nodes.

In [13]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "sharedstreets_extract", 
    "merced.shp"
)

# specify the column to look for county name

county_variable_name = 'NAME'

In [17]:
roadway_network.tidy_roadway(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name,
    create_node_link_id = False
)

2022-01-07 17:10:09, INFO: Starting Step 5 Tidy Roadway
2022-01-07 17:10:09, INFO: Joining network with county boundary file for ['Merced'] county
2022-01-07 17:10:34, INFO: Dropping links and nodes that are outside of ['Merced'] county
2022-01-07 17:10:34, INFO: Droppping circular links
2022-01-07 17:10:34, INFO: Flagging dead-end streets for drive network
2022-01-07 17:10:35, INFO: Making dead-end streets drive_access = 0
2022-01-07 17:10:37, INFO: Dropping alternative links between same AB nodes


## Step 6 - Build Transit Network from GTFS

Build standard transit network from GTFS

In [18]:
# the directory where all gtfs feeds are located
# e.g. gtfs_dir/sjrtd_2015_0127/routes.txt
# e.g. gtfs_dir/bart_2015_0127/routes.txt

gtfs_dir = os.path.join(external_dir, "gtfs", "2021")

In [37]:
# read gtfs into transit object

transit_network = Transit.load_all_gtfs_feeds(
    gtfs_dir = gtfs_dir,
    roadway_network= roadway_network,
    parameters=parameters
)

2022-01-07 18:13:00, INFO: Excluding weekend-only services
2022-01-07 18:13:00, INFO: Read and get representative transit feed from: D:/merced\data\external\gtfs\2021\The Bus
2022-01-07 18:13:01, INFO: Lasso base directory set as: D:/github/Ranch


In [38]:
# main activities - routing buses, creating rails

transit_network.build_standard_transit_network(
    multithread_shst_match = True,
    multithread_shortest_path = False
)

2022-01-07 18:13:02, INFO: Getting representative trip for each route by time of day and direction...
2022-01-07 18:13:02, INFO: Snapping gtfs stops to roadway node...
2022-01-07 18:13:05, INFO: Route bus trips using shortest path
2022-01-07 18:13:05, INFO: Routing bus on roadway network from start to end with osmnx...
2022-01-07 18:13:05, INFO: Setting good link dictionary
2022-01-07 18:13:09, INFO: 	Routing agency The Bus, trip t_1962833_b_33141_tn_1
2022-01-07 18:13:16, INFO: 	Routing agency The Bus, trip t_1962842_b_33141_tn_1
2022-01-07 18:13:23, INFO: 	Routing agency The Bus, trip t_1962805_b_33141_tn_8
2022-01-07 18:13:31, INFO: 	Routing agency The Bus, trip t_1962875_b_33141_tn_10
2022-01-07 18:13:38, INFO: 	Routing agency The Bus, trip t_1962843_b_33141_tn_18
2022-01-07 18:13:45, INFO: 	Routing agency The Bus, trip t_1962863_b_33141_tn_18
2022-01-07 18:13:51, INFO: 	Routing agency The Bus, trip t_1963111_b_33141_tn_0
2022-01-07 18:13:59, INFO: 	Routing agency The Bus, trip t_1

In [39]:
# check if shortest path transit builder failed any transit shapes

transit_network.shortest_path_failed_shape_list

['The Bus_p_3395']

In [40]:
transit_network.write_standard_transit(
    path = interim_dir
)

## Step 7 - Build Centroid Connectors

builds centroid connectors from TAZ, MAZ

In [ ]:
taz_polygon_file = os.path.join(
    external_dir,
    "taz",
    "SJ_TAZ_Aug2011.shp"
)

In [ ]:
roadway_network.build_centroid_connectors(
    build_taz_drive = True,
    build_taz_active_modes = True,
    input_taz_polygon_file = taz_polygon_file
)

## Step 8 - Write out Standard Format

In [41]:
roadway_network.standard_format(
    county_boundary_file = os.path.join(external_dir,
        "sharedstreets_extract", 
        "merced.shp"),
    county_variable_name = 'NAME'
)

2022-01-07 18:29:03, INFO: Starting Step 8 creating roadway standard format


In [42]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step8_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step8_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step8_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step8_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

2022-01-07 18:29:15, INFO: write out shape geojson
2022-01-07 18:29:20, INFO: write out node geojson
2022-01-07 18:29:23, INFO: write out link json
2022-01-07 18:29:24, INFO: write out link geojson
